<a href="https://skillfactory.ru/">
    <img src="https://raw.githubusercontent.com/dhegl/sf_ds/64c052f95af5d042844ed56f765c2cbb566d1680/main/static/medium.svg" alt="Онлайн-школа SkillFactory" width="160px" align="right" />
</a>

# Проект 2. Анализ вакансий по информации в базе данных (PostgreSQL)

*Анализ вакансий на основании данных, хранящихся в СУБД PostgreSQL, о вакансиях, работодателях и сферах их деятельности*

*Содержание*


*  Требования к оформлению учебной презентации
*  Решения и ответы на этапные вопросы по проекту в формате SQL запросов. 
    *  Выводы на основание полученных данных
*  Дополнительный анализ
*  Общий вывод

### Анализ данных о вакансиях и работодателях.

*Требования к оформлению*


Сделайте выводы не только на основе данных: попробуйте также проанализировать, за счёт чего получились именно такие цифры, постарайтесь увидеть тенденции, сделать прогнозы. Там, где это потребуется, проведите дополнительные исследования данных.

---

Необходимо сделать как отдельные выводы по результатам каждого блока заданий, так и один общий вывод по всем вакансиям, имеющимся в базе данных:

-  Каких вакансий много?
-  Что можно сказать о рынке труда в различных городах?
-  Какие ещё интересные исследования можно провести?

Вы можете добавить любые свои рассуждения и умозаключения, к которым пришли в процессе работы. 


Формат ноутбука с решением:

1.  Номер задания.
2.  Код для получения ответа.
3.  Результат запроса.
4.  Выводы по блоку заданий.
5.  Общий вывод в конце по результатам анализа, имеющихся данных по вакансиям.


### Установка и импорт необходимых модулей Python

In [1]:
## (!) Для работы с презентацией раскоментируйте следующий строки для установки нужных модулей

# %pip install sqlalchemy==1.4.46
# %pip install ipython-sql==0.4.1
# %pip install lxml==4.9.2

In [1]:
import pandas as pd
import sqlalchemy as sa
import psycopg2 as pg

## Работаем с ipython-sql расширением
#%load_ext sql
%reload_ext sql
## Отключить отображение подключений при запросе
%config SqlMagic.displaycon = False
## 
connections:dict

![WARNING](https://raw.githubusercontent.com/dhegl/sf_ds/main/main/static/warning.png)  


>**Данные авторизации, необходимые для подключения к базе данных учебной платформы удалены из презентации.**

*Для подключения необходимо указать значения для подключения к БД в следующих переменных в ячейке ниже:*  
```
DBNAME = ''
USER = ''
PASSWORD = ''
HOST = ''
PORT = 
```

In [2]:
### skillfactory database login
DBNAME = ''
USER = ''
PASSWORD = ''
HOST = ''
PORT = 5432


In [12]:
### При перезагрузке ноутбука удаляем действующие подключения %sql
### 
# connections = %sql -l
# ##print('connections:', connections)
# ##[c.session.close() for c in connections.values()]
# for cnn in connections.values(): 
#     cnn.session.close()
# connections.clear()

engine = sa.create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}') #localhost:5432
%sql $engine.url
### %sql postgresql://$USER:$PASSWORD@$HOST:$PORT/$DBNAME
%sql -l

#### Таблицы, поля и типы данных базы данных по вакансиям
```
areas
  id    integer	Идентификатор города
  name	text	Название города

employers
  id    integer	Номер работодателя
  name  text	Название работодателя
  area  integer	Регион регистрации

employers_industries
  employer_id	integer             ID работодателя
  industry_id	character varying   ID сферы деятельности

industries
  id    character varying   ID сферы деятельности
  name  text                Название сферы деятельности

vacancies
  id          integer ID вакансии
  name        text    Название вакансии
  area_id     integer ID региона вакансии
  employer_id integer ID работодателя
  employment  text    Тип трудоустройства
  experience  text    Требования к опыту
  key_skills  text    Ключевые навыки
  salary_from integer Нижняя граница зарплатной вилки
  salary_to   integer Верняя граница зарплатной вилки
  schedule    text    Тип рабочего графика
```

## Ответы на этапные задания модуля (проекта)

##### Задания 3.1 - 3.4

In [ ]:
%%sql

--  В виду достаточной наглядности и небольшой сложности всех запросов призентации,
-- комментарии SQL в запросах не используются, что бы не ухудшать визуальное оценку 
-- SQL кода проверяющим, за исключением закоментированных вариаций самого запроса.

SELECT      'areas' "table", COUNT(*) FROM areas      UNION
SELECT  'employers' "table", COUNT(*) FROM employers  UNION
SELECT  'vacancies' "table", COUNT(*) FROM vacancies  UNION
SELECT 'industries' "table", COUNT(*) FROM industries UNION
SELECT 'employers_industries' "table", COUNT(*) FROM employers_industries
-- ORDER BY "table"
ORDER BY count DESC;

5 rows affected.


table,count
vacancies,49197
employers_industries,32333
employers,23501
areas,1362
industries,294


Вывод

Количество данных, предоставленных в БД, достаточно для проведения исследования и базового анализа в рамках учебного проекта.

#####  Задание 4.1
  Выберите пятёрку лидеров по количеству вакансий из регионов

In [ ]:
%%sql

SELECT 
  a.name,
  COUNT(v.area_id) cnt
FROM vacancies v
JOIN areas a ON v.area_id = a.id
GROUP BY v.area_id, a.name
ORDER BY cnt DESC
LIMIT 5;


5 rows affected.


name,cnt
Москва,5333
Санкт-Петербург,2851
Минск,2112
Новосибирск,2006
Алматы,1892


Вывод

Ожидаемо, что столица будет лидировать по количеству востребованных вакансий. При несколько выделяющемся Санкт-Петербурге, в остальных 4-х крупных городах СНГ количество вакансий примерно одинаково.

##### Задание 4.2
  У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?

In [ ]:
%%sql

SELECT 
  count(*)
FROM vacancies v
WHERE v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL
-- WHERE NOT (v.salary_from IS NULL AND v.salary_to IS NULL)
;

1 rows affected.


count
24073


Вывод

Отметим, что только около 50% данных по вакансиям можно использовать для анализа предлагаемой заработной платы работодателем. Это довольно низкий, по критериям оценки пригодности для анализа, показатель для признака. Возможно требуется проведение дополнительной работы по получению более полных данных в разрезе других критериев(регион, сфера деятельности и т.д.)

#### Задание 4.3
  Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого числа.

In [ ]:
%%sql

SELECT 
  ROUND(AVG(v.salary_from),0) salary_avg_from, 
  ROUND(AVG(v.salary_to),0) salary_avg_to
FROM vacancies v;

1 rows affected.


salary_avg_from,salary_avg_to
71065,110537


Вывод

Можно заметить, показатели зарплатной вилки в среднем довольно большие для региональных зарплат. Скорее всего, из-за не полных данных от работодателей эти показатели завышены. 

#### Задание 4.4
  Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Какая пара находится на втором месте по популярности?


In [ ]:
%%sql

SELECT 
  v.schedule, 
  v.employment,
  COUNT(*)
FROM vacancies v
GROUP BY v.schedule,v.employment
ORDER BY COUNT DESC
LIMIT 5;

5 rows affected.


schedule,employment,count
Полный день,Полная занятость,35367
Удаленная работа,Полная занятость,7802
Гибкий график,Полная занятость,1593
Удаленная работа,Частичная занятость,1312
Сменный график,Полная занятость,940


Вывод

Полная занятость при полном рабочем дне абсолютный лидер по типу трудоустройства и рабочего графика среди остальных типов вакансий. А удаленная работа при сохранении полной занятости, стала фактически нормой во время и после пандемии для многих "офисных" сфер деятельности и является лишь разновидностью первой. Остальные типы сочетаний значительно отстают.

#### Задание 4.5  
Напишите запрос, выводящий значения поля Требуемый опыт работы (experience)
в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [ ]:
%%sql

SELECT 
  v.experience,
  COUNT(*)
FROM vacancies v
GROUP BY v.experience
ORDER BY COUNT;

4 rows affected.


experience,count
Более 6 лет,1337
Нет опыта,7197
От 3 до 6 лет,14511
От 1 года до 3 лет,26152


Вывод

По данным показателям видно, что основное требование по опыту от работодателя это что бы он был. Вакансий, где опыт не требуется, значительно меньше. Вакансии где указан опыт более 6 лет скорее всего относятся к лидерским позициям в сферах деятельности и составляют меньшинство.
Для более детальной оценки требований по опыту требуются дополнительные исследования в разрезе других критериальных признаков.

##### Задание 5.1
Напишите запрос, который позволит узнать, какие работодатели 
находятся на первом и пятом месте по количеству вакансий.


In [ ]:
%%sql

SELECT 
  COUNT(v.id),
  e.name
FROM vacancies v
LEFT JOIN employers e ON v.employer_id = e.id
GROUP BY e.name
ORDER BY COUNT DESC
LIMIT 5;

5 rows affected.


count,name
1933,Яндекс
491,Ростелеком
444,Тинькофф
428,СБЕР
331,Газпром нефть


Вывод

По абсолютно лидерским позициям Яндекса по количеству вакансиям можно судить, что компания серьезно развивается и нуждается в новых сотрудниках. У остального из "Топ5" российского бизнеса количество вакансий значительно ниже и находится примерно на одном уровне.

####  Задание 5.2 (Вариант второй, классический) 
Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём. Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей. Впишите его название в поле ниже в том виде, который вернул запрос.

In [ ]:
%%sql

SELECT 
  a.name,
  v.v_cnt,
  e.e_cnt
FROM areas a
LEFT JOIN (
    SELECT 
      e.area,
      COUNT(*) AS e_cnt
    FROM employers e
    GROUP BY e.area
  ) AS e ON a.id = e.area
LEFT JOIN (
    SELECT 
      v.area_id,
      COUNT(*) AS v_cnt
    FROM vacancies v
    GROUP BY v.area_id
  ) AS v ON a.id = v.area_id
-- WHERE v.v_cnt IS NULL
ORDER BY v.v_cnt DESC NULLS FIRST, e.e_cnt DESC NULLS LAST
LIMIT 5;

5 rows affected.


name,v_cnt,e_cnt
Россия,None,410
Казахстан,None,207
Московская область,None,75
Краснодарский край,None,19
Беларусь,None,18


#### Задание 5.2 (Вариант первый, исторический) 
  Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.
Впишите его название в поле ниже в том виде, который вернул запрос.


In [ ]:
%%sql

WITH area_empl_cnt AS 
(
SELECT 
  e.area,
  COUNT(*) employers_count
FROM employers e
GROUP BY e.area
)

SELECT
  a.name,
  COUNT(v.area_id) vacancies_count,
  e.employers_count
FROM areas a
LEFT JOIN vacancies v ON a.id = v.area_id
LEFT JOIN area_empl_cnt e ON e.area = a.id
GROUP BY a.id, a.name, e.employers_count
ORDER BY vacancies_count, e.employers_count DESC NULLS LAST
LIMIT 5;

5 rows affected.


name,vacancies_count,employers_count
Россия,0,410
Казахстан,0,207
Московская область,0,75
Краснодарский край,0,19
Беларусь,0,18


Вывод

Из "регионов-аггрегаторов" закономерно на первом и втором месте Россия и Казахстан, как страны с преобладающим количеством развивающихся направлений экономической и промышленной деятельности. Так же закономерно выделяется по количеству работодателей Московская область, как столичная. Остальные позиции значительно отстают.

#### Задание 5.3
  Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии
Выберите максимальное значение из получившегося списка.

In [ ]:
%%sql

SELECT
  e.name,
  COUNT(DISTINCT v.area_id)
FROM employers e 
LEFT JOIN vacancies v ON e.id = v.employer_id
GROUP BY e.id, e.name
ORDER BY COUNT DESC
LIMIT 5;

5 rows affected.


name,count
Яндекс,181
Ростелеком,152
Спецремонт,116
Поляков Денис Иванович,88
ООО ЕФИН,71


Вывод

Яндекс являясь наибольшим обладателем вакансий и здесь закономерно лидирует. Можно отметить, что вторая позиция Ростелекома связана со спецификой деятельности по предостовлению телекомуникационных услуг и востребованностью специалистов как для администрирования сетевого оборудования, так и для работы в "поле" по всей стране.

#### Задание 5.4  
  Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.
Введите количество, которое вернул запрос


In [ ]:
%%sql

SELECT  
  COUNT(*)
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
WHERE ei.employer_id IS NULL;

1 rows affected.


count
8419


Вывод

Показатель по отсутствующим данным довольно значительный, но возможно при конструировании признаков его удастся сократить за счет наименований вакансий. А причины, по которой работодатель не указал сферу деятельности могут быть весьма разнообразными.

#### Задание 5.5
  Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.
Введите в поле ниже название этой компании так же, как оно указано в результате запроса


In [ ]:
%%sql

SELECT  
  e.name,
  COUNT(ei) industry_cnt
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
GROUP BY e.id
HAVING COUNT(ei) = 4
ORDER BY e.name ASC
LIMIT 5;


5 rows affected.


name,industry_cnt
101 Интернет,4
21vek.by,4
2ГИС,4
2К,4
4 пикселя +,4


Вывод

Нет какой либо тенденции или закономерности для вывода, кроме повсеместной узнаваемости бренда "2ГИС".

#### Задание 5.6  
С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».


In [ ]:
%%sql

SELECT  
  count(*)
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
LEFT JOIN industries i ON ei.industry_id = i.id
WHERE i.name LIKE 'Разработка программного обеспечения';


1 rows affected.


count
3553


Вывод

Довольно большой относительный показатель для территории с очень разнообразной экономической деятельностью. Возможно это связанно с дефицитом специалистов данного направления. Требуются дополнительные иследования.

#### К Заданию 5.7. А.Загрузка списка городов из архива

In [ ]:
# Если нет доступа к wikipedia.org берем архивные данные
city_mills = [ \
'Москва',
'Санкт-Петербург',
'Новосибирск',
'Екатеринбург',
'Казань',
'Нижний Новгород',
'Челябинск',
'Красноярск',
'Самара',
'Уфа',
'Ростов-на-Дону',
'Омск',
'Краснодар',
'Воронеж',
'Пермь',
'Волгоград'
]

#### К Заданию 5.7. В.Загрузка и парсинг данных с сайта wikipedia.org

In [4]:
import urllib.parse as req

### Забираем города-миллионики с википедии
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
### и кладем в Serias
city_mills_ss = pd.read_html(url)[1]['Город']

In [5]:
## 
def remove_bracket(raw:str)->str:
    """Удаляет комментирующие ссылки в квадратных скобках"""
    bracket_counter, result = 0, ''
    for ch in raw:
        if ch == '[':
            bracket_counter += 1
        if ch == ']':
            bracket_counter -= 1
        elif not bracket_counter:
            result += ch
    return result

city_mills = city_mills_ss.apply(remove_bracket).to_list()

### Параметры для запроса
company_for_select = 'Яндекс'
city_params = tuple(city_mills)
print(company_for_select, '::', ', '.join(city_mills))

Яндекс :: Москва, Санкт-Петербург, Новосибирск, Екатеринбург, Казань, Нижний Новгород, Челябинск, Красноярск, Самара, Уфа, Ростов-на-Дону, Омск, Краснодар, Воронеж, Пермь, Волгоград


#### *Задание 5.7
  Для компании «Яндекс» выведите список регионов-миллионников , в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании, собранных в этой таблице.


In [ ]:
%%sql

(
  SELECT 
    -- 'Total' area_name,
    '*Всего вакансий' area_name,
    COUNT(*) vacancies_count
  FROM vacancies v
  LEFT JOIN employers e ON v.employer_id = e.id
  JOIN areas a ON v.area_id = a.id
  WHERE e.name LIKE :company_for_select
  AND a.name IN :city_params
  GROUP BY v.employer_id
)
UNION ALL
(
  SELECT 
    a.name area_name,
    COUNT(*) vacancies_count
  FROM vacancies v
  LEFT JOIN employers e ON v.employer_id = e.id
  JOIN areas a ON v.area_id = a.id
  WHERE e.name LIKE :company_for_select
  AND a.name IN :city_params
  GROUP BY v.area_id, v.employer_id, a.name
)
ORDER BY vacancies_count DESC;


17 rows affected.


area_name,vacancies_count
*Всего вакансий,485
Москва,54
Санкт-Петербург,42
Екатеринбург,39
Нижний Новгород,36
Новосибирск,35
Воронеж,32
Краснодар,30
Самара,26
Уфа,26


Вывод

Каких либо аномалий по данной выборке не наблюдается. Столицы закономерно лидируют по количеству вакансий от IT компании.

#### Задание 6.1
  Сколько вакансий имеют отношение к данным ('data', 'данн')


In [ ]:
%%sql

SELECT  
  COUNT(*)
FROM vacancies v 
WHERE v.name iLIKE '%data%' OR v.name iLIKE '%данн%';

1 rows affected.


count
1771


Вывод

Можно отметить, что показатель довольно высокий. Связанно это скорее с тем, что под работой с данными подразумевается довольно широкий спектр деятельности. 

#### Задание 6.2  
Сколько есть подходящих вакансий для начинающего дата-сайентиста?

In [ ]:
%%sql

WITH x AS (
SELECT  
  *
FROM vacancies v 
WHERE 
    v.name iLIKE '%data scientist%' 
 OR v.name iLIKE '%data science%'
 OR v.name iLIKE '%исследователь данных%' 
 OR v.name iLIKE '%machine learning%' 
 OR v.name iLIKE '%машинн%обучен%'
 OR v.name LIKE '%ML%' AND NOT v.name LIKE '%HTML%'
)

SELECT 
  COUNT(*)
FROM x
WHERE 
     x.name iLIKE '%junior%'
  OR x.experience LIKE 'Нет опыта'
  OR x.employment LIKE 'Стажировка';

1 rows affected.


count
51


Вывод

Удручающий. Практически отсутствие вакансий для молодых специалистов и специалистов уровня "junior" при росте востребованности в области обработки больших данных. Нежелание компаний и учреждений предлагать рабочие места и места стажировок начинающим специалистам данного направления и "растить" специалистов "под себя", будет сказываться на количественном, а в последствии и качественном развитии этой сферы.

####  Задание 6.3  
Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

In [ ]:
%%sql

WITH x AS (
SELECT  
  *
FROM vacancies v 
WHERE 
    v.name iLIKE '%data scientist%' 
 OR v.name iLIKE '%data science%'
 OR v.name iLIKE '%исследователь данных%' 
 OR v.name iLIKE '%machine learning%' 
 OR v.name iLIKE '%машинн%обучен%'
 OR v.name LIKE '%ML%' AND NOT v.name LIKE '%HTML%'
)

SELECT 
  COUNT(*)
FROM x
WHERE 
    x.key_skills iLIKE '%sql%'
 OR x.key_skills iLIKE '%postgres%';

1 rows affected.


count
201


Вывод

Можно лишь отметить, что данное направление (DS) зачастую связанно с непосредственной работой с хранилищами данных в виде реляционных СУБД SQL.

####  Задание 6.4  
С помощью запроса, аналогичного предыдущему, проверьте, насколько популярен Python в требованиях работодателей к DS. Вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

In [ ]:
%%sql

WITH x AS (
SELECT  
  *
FROM vacancies v 
WHERE 
    v.name iLIKE '%data scientist%' 
 OR v.name iLIKE '%data science%'
 OR v.name iLIKE '%исследователь данных%' 
 OR v.name iLIKE '%machine learning%' 
 OR v.name iLIKE '%машинн%обучен%'
 OR v.name LIKE '%ML%' AND NOT v.name LIKE '%HTML%'
)

SELECT 
  COUNT(*)
FROM x
WHERE 
    x.key_skills iLIKE '%python%'
-- AND x.key_skills iLIKE '%postgres%'
;

1 rows affected.


count
351


Вывод

На фоне огромого разнообразия различных инструментов и готовых решений для анализа данных, можно отметит, что популярность Python довольно высокая. Связано это со многими факторами, включая open-source лецензирование, низкий порог входа для освоения языка и инструментария.  Ну и несомненно популярность Python как языка с большим количеством разнообразных библиотек, средств разработки, готовых фрэймверков и их сфер приминения от агентов и терминалов мониторинга, инструментальных средст разработки и иследования до бэкэнд ядра крупного информационного или аналитического web портала. Весь процесс от иследования и разработки, до развертывания в сети и сопровождения из одной "коробки" и как следствие - снижение стоимости разработки и сопровождения.

####  Задание 6.5 (Прототип запроса)  
Сколько ключевых навыков в среднем указывают в вакансиях для DS? Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
%%sql

WITH x AS (
SELECT  
  *
FROM vacancies v 
WHERE 
    v.name iLIKE '%data scientist%' 
 OR v.name iLIKE '%data science%'
 OR v.name iLIKE '%исследователь данных%' 
 OR v.name iLIKE '%machine learning%' 
 OR v.name iLIKE '%машинн%обучен%'
 OR v.name LIKE '%ML%' AND NOT v.name LIKE '%HTML%'
)

SELECT 
  x.key_skills,
  ARRAY_LENGTH(string_to_array(x.key_skills, chr(9)), 1)
FROM x
LIMIT 5;

5 rows affected.


key_skills,array_length
Python SQL Работа в команде OpenCV ML,5
Oracle Pl/SQL Базы данных Machine Learning Машинное обучение Git Data science Python Работа с базами данных Математические модели,9
Python OpenCV Tensorflow Computer Vision Нейронные сети C++ С# Caffe,8
Python Flask Linux AWS Git,5
Python SQL NLP Scikit-learn Machine Learning,5


#### Задание 6.5 (Решение)  
Сколько ключевых навыков в среднем указывают в вакансиях для DS? Ответ округлите до двух знаков после точки-разделителя.


In [ ]:
%%sql

WITH x AS (
SELECT  
  *
FROM vacancies v 
WHERE 
    v.name iLIKE '%data scientist%' 
 OR v.name iLIKE '%data science%'
 OR v.name iLIKE '%исследователь данных%' 
 OR v.name iLIKE '%machine learning%' 
 OR v.name iLIKE '%машинн%обучен%'
 OR v.name LIKE '%ML%' AND NOT v.name LIKE '%HTML%'
)

SELECT 
  ROUND(AVG(ARRAY_LENGTH(string_to_array(x.key_skills, chr(9)), 1)),2) "ds_skills_num_avg"
FROM x;

1 rows affected.


ds_skills_num_avg
6.41


Вывод

Требования работодателя к специалистам DS по наличию в среднем 6-7 навыков вполне закономерно учитывая с каким большим спектром программных и инструментальных средств приходится работать в этой области. По факту, с учетом наукоемкости и конвергентности этой специализации, этот показатель даже занижен.

#### Задание 6.6 A Вариант 1  
Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля experience).


In [ ]:
%%sql

WITH x AS (
  SELECT  
    *
  FROM vacancies v 
  WHERE 
      v.name iLIKE '%data scientist%' 
   OR v.name iLIKE '%data science%'
   OR v.name iLIKE '%исследователь данных%' 
   OR v.name iLIKE '%machine learning%' 
   OR v.name iLIKE '%машинн%обучен%'
   OR v.name LIKE '%ML%' AND NOT v.name LIKE '%HTML%'
)

SELECT 
  x.experience,
  ROUND(AVG(x.salary_from)) salary_avg_from,
  ROUND(AVG(x.salary_to)) salary_avg_to, 
  ROUND(AVG((COALESCE(x.salary_from, x.salary_to) + COALESCE(x.salary_to, x.salary_from))/2)) salary_avg
FROM x
WHERE x.salary_from IS NOT NULL OR x.salary_to IS NOT NULL
GROUP BY x.experience;


3 rows affected.


experience,salary_avg_from,salary_avg_to,salary_avg
От 3 до 6 лет,198949,290589,243115
От 1 года до 3 лет,121272,175673,139675
Нет опыта,61000,97000,74643


#### Задание 6.6 A Вариант 2 

In [ ]:
%%sql

SELECT 
  x.experience,
  ROUND(AVG(x.salary_from)) salary_avg_from,
  ROUND(AVG(x.salary_to)) salary_avg_to, 
  ROUND(AVG((COALESCE(x.salary_from, x.salary_to) + COALESCE(x.salary_to, x.salary_from))/2)) salary_avg
FROM (
  SELECT  
    *
  FROM vacancies v 
  WHERE 
      v.name iLIKE '%data scientist%' 
  OR v.name iLIKE '%data science%'
  OR v.name iLIKE '%исследователь данных%' 
  OR v.name iLIKE '%machine learning%' 
  OR v.name iLIKE '%машинн%обучен%'
  OR v.name LIKE '%ML%' AND NOT v.name LIKE '%HTML%'
) as x
WHERE x.salary_from IS NOT NULL OR x.salary_to IS NOT NULL
GROUP BY x.experience;


3 rows affected.


experience,salary_avg_from,salary_avg_to,salary_avg
Нет опыта,61000,97000,74643
От 1 года до 3 лет,121272,175673,139675
От 3 до 6 лет,198949,290589,243115


####  Задание 6.6 B Вариант 3
Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. 
Результат округлите до целого числа.

In [ ]:
%%sql

SELECT 
  x.experience,
  ROUND(AVG(x.salary_from)) salary_avg_from,
  ROUND(AVG(x.salary_to)) salary_avg_to, 
  ROUND(AVG((COALESCE(x.salary_from, x.salary_to) + COALESCE(x.salary_to, x.salary_from))/2)) salary_avg
FROM (
  SELECT  
    *
  FROM vacancies v 
  WHERE v.name ILIKE ANY (
    VALUES ('%data scientist%'),
           ('%data science%'),
           ('%исследователь данных%'),
           ('%machine learning%'),
           ('%машинн%обучен%')
  ) OR v.name LIKE '%ML%' AND NOT v.name LIKE '%HTML%'
) as x
WHERE x.salary_from IS NOT NULL OR x.salary_to IS NOT NULL
GROUP BY x.experience
HAVING x.experience LIKE 'От 3 до 6 лет';


1 rows affected.


experience,salary_avg_from,salary_avg_to,salary_avg
От 3 до 6 лет,198949,290589,243115


Вывод

Довольно высокий уровень средней зарплаты специалистов из области DS с опытом работы от 3 до 6 лет. С увереностью можно лишь сказать, что и требования к данным специалистам будут такими же высокими.

---

### Дополнительный анализ данных о вакансиях и работодателях.


#### Топ 10 типов вакансий

Распределение количества вокансий по популярности

In [ ]:
%%sql

SELECT  
  v.name,
  COUNT(*)
FROM vacancies v 
GROUP BY v.name
ORDER BY COUNT DESC
LIMIT 10;

10 rows affected.


name,count
Системный администратор,1761
Программист 1С,1153
Бизнес-аналитик,488
Специалист технической поддержки,457
Системный аналитик,432
Аналитик,404
Инженер-программист,389
Программист 1C,360
Frontend-разработчик,242
Инженер технической поддержки,230


Вывод

Так как формулировка "Системный администратор" предполагает довольно широкий круг деятельности и является собирательной для разнообразных как систем так и подходов к администрированию и зачастую является стартовым направлением в другие, более специализированные направления в IT, то с уверенностью можно сказать, что по более узкой специализации наибольшее количество вакансий в позиции "Программист 1С", что и подтверждается фактами о дефиците данных специалистов и "перегретых" зарплатах. Связанно это с фактической монопольностью продуктов фирмы "1С" на рынке в востребованном сегменте финансового и бухгалтерского п/о.

#### Сфера образования

Количество вакансий для преподователей и учителей

In [ ]:
%%sql

SELECT  
  v.name,
  COUNT(*)
FROM vacancies v 
WHERE v.name ILIKE '%преподователь%' 
   OR v.name ILIKE '%учитель%'
   OR v.name ILIKE '%инструктор%'
   OR v.name ILIKE '%ментор%'
GROUP BY v.name
ORDER BY COUNT DESC
LIMIT 15;


12 rows affected.


name,count
Учитель информатики,8
Junior+/Middle Go Разработчик - Ментор,1
Python разработчик (ментор),1
Водитель - инструктор категории Е к С,1
Ментор курса «Разработка и продвижение Web-проектов»,1
Ментор курса Разработка программного обеспечения,1
"Ментор на курс (React, Python)",1
Преподаватель/учитель школы дизайна и архитектуры,1
Учитель информатики и программирования,1
Учитель информатики/программирования,1


Вывод

Как видим, на фоне того, что  в сфере образования вакансий крайне мало, выделяется позицию "Учитель информатики", что говорит о либо о нехватке данных преподавателей, либо о низкой заработной плате по данной позиции. А скорее всего и того и другого одновременно. Бизнес сфера для специалистов по информатике предлагает лучшее условия, чем образовательная деятельность в школе. 

#### Регионы-миллионники
  Список регионов-миллионников с количеством вакансий в этих регионах.

In [11]:
%%sql

(
  SELECT 
    -- 'Total' area_name,
    '*Всего вакансий' area_name,
    COUNT(*) vacancies_count
  FROM vacancies v
  JOIN areas a ON v.area_id = a.id
  WHERE a.name IN :city_params
)
UNION ALL
(
  SELECT 
    a.name area_name,
    COUNT(*) vacancies_count
  FROM vacancies v
  JOIN areas a ON v.area_id = a.id
  WHERE a.name IN :city_params
  GROUP BY v.area_id, a.name
)
ORDER BY vacancies_count DESC;

17 rows affected.


area_name,vacancies_count
*Всего вакансий,23856
Москва,5333
Санкт-Петербург,2851
Новосибирск,2006
Екатеринбург,1698
Нижний Новгород,1670
Казань,1415
Краснодар,1301
Самара,1144
Ростов-на-Дону,1131


Вывод

Около 50% всех вакансий сосредоточенно в городах-миллиониках. Столица ожидаемо лидирует, остальные значения по количеству соответствуют размеру региона и удаленности от основного бизнес центра страны.

##  Закрытие подключения к БД

In [ ]:
### Закрываем подключения %sql
###
connections = %sql -l
for cnn in connections.values(): 
    cnn.session.close()
connections.clear()

# Общий вывод

-  Функционал DDL и DML современных реляционных СУБД, различные OLAP расширения языка запросов SQL позволяют провести большую часть работы по подготовке и анализу данных, выявлению основных и значимых признаков для машинного обучения. 

-  Знание языка запросов SQL в контексте работы с реляционными СУБД, умение применять его при обработке и анализе больших данных, а так же умения оптимизировать запросы SQL может дать значительные конкурентные преимущества при карьерном росте на старте в позиции специалиста в области Data Scince. И не только на старте. Реляционные базы стали практическим стандартом для хранения и обработки малых и больших массивов данных в информационных и технологических системах от встраиваемых СУБД типа SQLite, различных "настольных" в составе разнообразных офисных пакетов, интеграционных универсалов MySQL и PostgreSQL, до гигантских кластеров данных на базе решений Oracle, DB2 и MSSQL.

-  Не смотря на то, что с помощью конструкций языка SQL можно провести большую подготовительную работу по предварительному анализу данных, находящихся в хранилищах современных реляционных СУБД, до того как данные будут готовы для построения модели требуется затратить еще много времени и сил для их подготовке к моделированию и машинному обучению.




*SF DST-148 Дмитрий Орлов Февраль 2023*

<a href="https://skillfactory.ru/courses/data-science">
    <img src="https://raw.githubusercontent.com/dhegl/sf_ds/64c052f95af5d042844ed56f765c2cbb566d1680/main/static/small.svg" alt="Онлайн-школа SkillFactory Курсы по Data Science" width="120px" align="left" >
</a>